In [1]:
import os
import scipy.misc
import numpy as np
import json

# from model import DCGAN
from model_steer2 import DCGAN
from utils import pp, visualize, to_json, show_all_variables, expand_path, timestamp

import tensorflow as tf

flags = tf.app.flags
flags.DEFINE_integer("epoch", 25, "Epoch to train [25]")
flags.DEFINE_float("learning_rate", 0.0002, "Learning rate of for adam [0.0002]")
flags.DEFINE_float("beta1", 0.5, "Momentum term of adam [0.5]")
flags.DEFINE_float("train_size", np.inf, "The size of train images [np.inf]")
flags.DEFINE_integer("batch_size", 64, "The size of batch images [64]")
flags.DEFINE_integer("input_height", 108, "The size of image to use (will be center cropped). [108]")
flags.DEFINE_integer("input_width", None, "The size of image to use (will be center cropped). If None, same value as input_height [None]")
flags.DEFINE_integer("output_height", 64, "The size of the output images to produce [64]")
flags.DEFINE_integer("output_width", None, "The size of the output images to produce. If None, same value as output_height [None]")
flags.DEFINE_string("dataset", "mnist", "The name of dataset [celebA, mnist, lsun]")
flags.DEFINE_string("input_fname_pattern", "*.jpg", "Glob pattern of filename of input images [*]")
flags.DEFINE_string("data_dir", "./data", "path to datasets [e.g. $HOME/data]")
flags.DEFINE_string("out_dir", "./out", "Root directory for outputs [e.g. $HOME/out]")
flags.DEFINE_string("out_name", "", "Folder (under out_root_dir) for all outputs. Generated automatically if left blank []")
flags.DEFINE_string("checkpoint_dir", "checkpoint", "Folder (under out_root_dir/out_name) to save checkpoints [checkpoint]")
flags.DEFINE_string("sample_dir", "samples", "Folder (under out_root_dir/out_name) to save samples [samples]")
flags.DEFINE_boolean("train", False, "True for training, False for testing [False]")
flags.DEFINE_boolean("crop", False, "True for training, False for testing [False]")
flags.DEFINE_boolean("visualize", False, "True for visualizing, False for nothing [False]")
flags.DEFINE_boolean("export", False, "True for exporting with new batch size")
flags.DEFINE_boolean("freeze", False, "True for exporting with new batch size")
flags.DEFINE_integer("max_to_keep", 1, "maximum number of checkpoints to keep")
flags.DEFINE_integer("sample_freq", 200, "sample every this many iterations")
flags.DEFINE_integer("ckpt_freq", 200, "save checkpoint every this many iterations")
flags.DEFINE_integer("z_dim", 100, "dimensions of z")
flags.DEFINE_string("z_dist", "uniform_signed", "'normal01' or 'uniform_unsigned' or uniform_signed")
flags.DEFINE_boolean("G_img_sum", False, "Save generator image summaries in log")
#flags.DEFINE_integer("generate_test_images", 100, "Number of images to generate during test. [100]")
FLAGS = flags.FLAGS

In [2]:
def main(_):
  pp.pprint(flags.FLAGS.__flags)
  
  # expand user name and environment variables
  FLAGS.data_dir = expand_path(FLAGS.data_dir)
  FLAGS.out_dir = expand_path(FLAGS.out_dir)
  FLAGS.out_name = expand_path(FLAGS.out_name)
  FLAGS.checkpoint_dir = expand_path(FLAGS.checkpoint_dir)
  FLAGS.sample_dir = expand_path(FLAGS.sample_dir)

  print('FLAGS.dataset:', FLAGS.dataset)  
  print('FLAGS.output_height', FLAGS.output_height)  
  print('flags.train', FLAGS.train)

  if FLAGS.output_height is None: FLAGS.output_height = FLAGS.input_height
  if FLAGS.input_width is None: FLAGS.input_width = FLAGS.input_height
  if FLAGS.output_width is None: FLAGS.output_width = FLAGS.output_height

  # output folders
  if FLAGS.out_name == "":
      FLAGS.out_name = '{} - {} - {}'.format(timestamp(), FLAGS.data_dir.split('/')[-1], FLAGS.dataset) # penultimate folder of path
      if FLAGS.train:
        FLAGS.out_name += ' - x{}.z{}.{}.y{}.b{}'.format(FLAGS.input_width, FLAGS.z_dim, FLAGS.z_dist, FLAGS.output_width, FLAGS.batch_size)

  FLAGS.out_dir = os.path.join(FLAGS.out_dir, FLAGS.out_name)
  FLAGS.checkpoint_dir = os.path.join(FLAGS.out_dir, FLAGS.checkpoint_dir)
  FLAGS.sample_dir = os.path.join(FLAGS.out_dir, FLAGS.sample_dir)

  if not os.path.exists(FLAGS.checkpoint_dir): os.makedirs(FLAGS.checkpoint_dir)
  if not os.path.exists(FLAGS.sample_dir): os.makedirs(FLAGS.sample_dir)

  with open(os.path.join(FLAGS.out_dir, 'FLAGS.json'), 'w') as f:
    flags_dict = {k:FLAGS[k].value for k in FLAGS}
    json.dump(flags_dict, f, indent=4, sort_keys=True, ensure_ascii=False)
  

  #gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.333)
  run_config = tf.ConfigProto()
  run_config.gpu_options.allow_growth=True

  with tf.Session(config=run_config) as sess:
    if FLAGS.dataset == 'mnist':
      dcgan = DCGAN(
          sess,
          input_width=FLAGS.input_width,
          input_height=FLAGS.input_height,
          output_width=FLAGS.output_width,
          output_height=FLAGS.output_height,
          batch_size=FLAGS.batch_size,
          sample_num=FLAGS.batch_size,
          y_dim=10,
          z_dim=FLAGS.z_dim,
          dataset_name=FLAGS.dataset,
          input_fname_pattern=FLAGS.input_fname_pattern,
          crop=FLAGS.crop,
          checkpoint_dir=FLAGS.checkpoint_dir,
          sample_dir=FLAGS.sample_dir,
          data_dir=FLAGS.data_dir,
          out_dir=FLAGS.out_dir,
          max_to_keep=FLAGS.max_to_keep)
    else:
      dcgan = DCGAN(
          sess,
          input_width=FLAGS.input_width,
          input_height=FLAGS.input_height,
          output_width=FLAGS.output_width,
          output_height=FLAGS.output_height,
          batch_size=FLAGS.batch_size,
          sample_num=FLAGS.batch_size,
          z_dim=FLAGS.z_dim,
          dataset_name=FLAGS.dataset,
          input_fname_pattern=FLAGS.input_fname_pattern,
          crop=FLAGS.crop,
          checkpoint_dir=FLAGS.checkpoint_dir,
          sample_dir=FLAGS.sample_dir,
          data_dir=FLAGS.data_dir,
          out_dir=FLAGS.out_dir,
          max_to_keep=FLAGS.max_to_keep)

    show_all_variables()

    if FLAGS.train:
      dcgan.train(FLAGS)
    else:
      load_success, load_counter = dcgan.load(FLAGS.checkpoint_dir)
      if not load_success:
        raise Exception("Checkpoint not found in " + FLAGS.checkpoint_dir)


    # to_json("./web/js/layers.js", [dcgan.h0_w, dcgan.h0_b, dcgan.g_bn0],
    #                 [dcgan.h1_w, dcgan.h1_b, dcgan.g_bn1],
    #                 [dcgan.h2_w, dcgan.h2_b, dcgan.g_bn2],
    #                 [dcgan.h3_w, dcgan.h3_b, dcgan.g_bn3],
    #                 [dcgan.h4_w, dcgan.h4_b, None])

    # Below is codes for visualization
      if FLAGS.export:
        export_dir = os.path.join(FLAGS.checkpoint_dir, 'export_b'+str(FLAGS.batch_size))
        dcgan.save(export_dir, load_counter, ckpt=True, frozen=False)

      if FLAGS.freeze:
        export_dir = os.path.join(FLAGS.checkpoint_dir, 'frozen_b'+str(FLAGS.batch_size))
        dcgan.save(export_dir, load_counter, ckpt=False, frozen=True)

      if FLAGS.visualize:
        OPTION = 1
        visualize(sess, dcgan, FLAGS, OPTION, FLAGS.sample_dir)

# if __name__ == '__main__':
#   tf.app.run()

In [3]:
tf.app.run(argv=['--dataset', 'mnist', '--input_height=28', '--output_height=28', '--train', '--out_name=ckpt'])

{'G_img_sum': <absl.flags._flag.BooleanFlag object at 0x7fee70e37ba8>,
 'batch_size': <absl.flags._flag.Flag object at 0x7fef49b376d8>,
 'beta1': <absl.flags._flag.Flag object at 0x7fef49b37630>,
 'checkpoint_dir': <absl.flags._flag.Flag object at 0x7fee70e374a8>,
 'ckpt_freq': <absl.flags._flag.Flag object at 0x7fee70e378d0>,
 'crop': <absl.flags._flag.BooleanFlag object at 0x7fee70e376a0>,
 'data_dir': <absl.flags._flag.Flag object at 0x7fee70e372e8>,
 'dataset': <absl.flags._flag.Flag object at 0x7fee70e37160>,
 'epoch': <absl.flags._flag.Flag object at 0x7feed17b9cf8>,
 'export': <absl.flags._flag.BooleanFlag object at 0x7fee70e377b8>,
 'freeze': <absl.flags._flag.BooleanFlag object at 0x7fee70e37828>,
 'h': <tensorflow.python.platform.app._HelpFlag object at 0x7fef5dfb65f8>,
 'help': <tensorflow.python.platform.app._HelpFlag object at 0x7fef5dfb65f8>,
 'helpfull': <tensorflow.python.platform.app._HelpfullFlag object at 0x7fef5dfb6668>,
 'helpshort': <tensorflow.python.platform.app

[      37 Epoch:[ 0/25] [  36/1093] time: 8.2744, d_loss: 1.50534177, g_loss: 2.20563841, walk_loss: 2.20563865
[      38 Epoch:[ 0/25] [  37/1093] time: 8.3659, d_loss: 1.50192845, g_loss: 2.21674657, walk_loss: 2.21674657
[      39 Epoch:[ 0/25] [  38/1093] time: 8.4554, d_loss: 1.50005329, g_loss: 2.19074488, walk_loss: 2.19074512
[      40 Epoch:[ 0/25] [  39/1093] time: 8.5505, d_loss: 1.50070918, g_loss: 2.20074606, walk_loss: 2.20074606
[      41 Epoch:[ 0/25] [  40/1093] time: 8.6424, d_loss: 1.49744844, g_loss: 2.18671227, walk_loss: 2.18671227
[      42 Epoch:[ 0/25] [  41/1093] time: 8.7329, d_loss: 1.50308037, g_loss: 2.20380974, walk_loss: 2.20380974
[      43 Epoch:[ 0/25] [  42/1093] time: 8.8223, d_loss: 1.50877249, g_loss: 2.22900701, walk_loss: 2.22900701
[      44 Epoch:[ 0/25] [  43/1093] time: 8.9143, d_loss: 1.49881911, g_loss: 2.19320130, walk_loss: 2.19320130
[      45 Epoch:[ 0/25] [  44/1093] time: 9.0072, d_loss: 1.50578582, g_loss: 2.22516036, walk_loss: 2.2

[     112 Epoch:[ 0/25] [ 111/1093] time: 15.0873, d_loss: 1.50283575, g_loss: 2.18416262, walk_loss: 2.18416262
[     113 Epoch:[ 0/25] [ 112/1093] time: 15.1707, d_loss: 1.50214398, g_loss: 2.18674135, walk_loss: 2.18674135
[     114 Epoch:[ 0/25] [ 113/1093] time: 15.2537, d_loss: 1.51915240, g_loss: 2.21669674, walk_loss: 2.21669674
[     115 Epoch:[ 0/25] [ 114/1093] time: 15.3393, d_loss: 1.50646448, g_loss: 2.19077873, walk_loss: 2.19077873
[     116 Epoch:[ 0/25] [ 115/1093] time: 15.4246, d_loss: 1.50659990, g_loss: 2.20056677, walk_loss: 2.20056677
[     117 Epoch:[ 0/25] [ 116/1093] time: 15.5116, d_loss: 1.50647807, g_loss: 2.19721341, walk_loss: 2.19721341
[     118 Epoch:[ 0/25] [ 117/1093] time: 15.6017, d_loss: 1.49924350, g_loss: 2.18098402, walk_loss: 2.18098426
[     119 Epoch:[ 0/25] [ 118/1093] time: 15.6951, d_loss: 1.50684142, g_loss: 2.21099043, walk_loss: 2.21099043
[     120 Epoch:[ 0/25] [ 119/1093] time: 15.7820, d_loss: 1.50767374, g_loss: 2.21017838, walk_

[     187 Epoch:[ 0/25] [ 186/1093] time: 21.7681, d_loss: 1.50709009, g_loss: 2.23055601, walk_loss: 2.23055601
[     188 Epoch:[ 0/25] [ 187/1093] time: 21.8544, d_loss: 1.50744295, g_loss: 2.22730517, walk_loss: 2.22730517
[     189 Epoch:[ 0/25] [ 188/1093] time: 21.9390, d_loss: 1.50945079, g_loss: 2.22185278, walk_loss: 2.22185278
[     190 Epoch:[ 0/25] [ 189/1093] time: 22.0249, d_loss: 1.50338805, g_loss: 2.20613980, walk_loss: 2.20613956
[     191 Epoch:[ 0/25] [ 190/1093] time: 22.1144, d_loss: 1.50293875, g_loss: 2.18920040, walk_loss: 2.18920040
[     192 Epoch:[ 0/25] [ 191/1093] time: 22.1996, d_loss: 1.49504054, g_loss: 2.19274259, walk_loss: 2.19274259
[     193 Epoch:[ 0/25] [ 192/1093] time: 22.2835, d_loss: 1.49717474, g_loss: 2.18790078, walk_loss: 2.18790102
[     194 Epoch:[ 0/25] [ 193/1093] time: 22.3736, d_loss: 1.49148393, g_loss: 2.18757153, walk_loss: 2.18757153
[     195 Epoch:[ 0/25] [ 194/1093] time: 22.4632, d_loss: 1.50313270, g_loss: 2.21193719, walk_

[     261 Epoch:[ 0/25] [ 260/1093] time: 32.1956, d_loss: 1.50612247, g_loss: 2.20133805, walk_loss: 2.20133805
[     262 Epoch:[ 0/25] [ 261/1093] time: 32.2830, d_loss: 1.50428033, g_loss: 2.18594670, walk_loss: 2.18594694
[     263 Epoch:[ 0/25] [ 262/1093] time: 32.3739, d_loss: 1.51569605, g_loss: 2.22196722, walk_loss: 2.22196722
[     264 Epoch:[ 0/25] [ 263/1093] time: 32.4639, d_loss: 1.51046896, g_loss: 2.21719766, walk_loss: 2.21719766
[     265 Epoch:[ 0/25] [ 264/1093] time: 32.5506, d_loss: 1.51378918, g_loss: 2.22013903, walk_loss: 2.22013903
[     266 Epoch:[ 0/25] [ 265/1093] time: 32.6397, d_loss: 1.49900675, g_loss: 2.20222807, walk_loss: 2.20222807
[     267 Epoch:[ 0/25] [ 266/1093] time: 32.7341, d_loss: 1.50651515, g_loss: 2.19298053, walk_loss: 2.19298053
[     268 Epoch:[ 0/25] [ 267/1093] time: 32.8243, d_loss: 1.49353063, g_loss: 2.19876075, walk_loss: 2.19876075
[     269 Epoch:[ 0/25] [ 268/1093] time: 32.9107, d_loss: 1.50460076, g_loss: 2.18846655, walk_

...

[   27142 Epoch:[24/25] [ 909/1093] time: 2986.9419, d_loss: 1.28255677, g_loss: 2.26119041, walk_loss: 2.26119041
[   27143 Epoch:[24/25] [ 910/1093] time: 2987.0308, d_loss: 1.21553266, g_loss: 2.28885722, walk_loss: 2.28885770
[   27144 Epoch:[24/25] [ 911/1093] time: 2987.1163, d_loss: 1.22050035, g_loss: 2.24774098, walk_loss: 2.24774098
[   27145 Epoch:[24/25] [ 912/1093] time: 2987.2043, d_loss: 1.29635525, g_loss: 2.28765440, walk_loss: 2.28765416
[   27146 Epoch:[24/25] [ 913/1093] time: 2987.2933, d_loss: 1.29161537, g_loss: 2.38481450, walk_loss: 2.38481474
[   27147 Epoch:[24/25] [ 914/1093] time: 2987.3848, d_loss: 1.24772894, g_loss: 2.28712606, walk_loss: 2.28712606
[   27148 Epoch:[24/25] [ 915/1093] time: 2987.4809, d_loss: 1.21418500, g_loss: 2.40427780, walk_loss: 2.40427780
[   27149 Epoch:[24/25] [ 916/1093] time: 2987.5722, d_loss: 1.20530128, g_loss: 2.36921620, walk_loss: 2.36921644
[   27150 Epoch:[24/25] [ 917/1093] time: 2987.6596, d_loss: 1.25032222, g_loss:

[   27215 Epoch:[24/25] [ 982/1093] time: 2997.7893, d_loss: 1.22010529, g_loss: 2.25809288, walk_loss: 2.25809288
[   27216 Epoch:[24/25] [ 983/1093] time: 2997.8775, d_loss: 1.18874645, g_loss: 2.30800748, walk_loss: 2.30800748
[   27217 Epoch:[24/25] [ 984/1093] time: 2997.9674, d_loss: 1.14888489, g_loss: 2.45768785, walk_loss: 2.45768785
[   27218 Epoch:[24/25] [ 985/1093] time: 2998.0604, d_loss: 1.22733307, g_loss: 2.30832171, walk_loss: 2.30832171
[   27219 Epoch:[24/25] [ 986/1093] time: 2998.1447, d_loss: 1.23104787, g_loss: 2.33341241, walk_loss: 2.33341193
[   27220 Epoch:[24/25] [ 987/1093] time: 2998.2300, d_loss: 1.23433793, g_loss: 2.32939816, walk_loss: 2.32939839
[   27221 Epoch:[24/25] [ 988/1093] time: 2998.3181, d_loss: 1.21580958, g_loss: 2.37298322, walk_loss: 2.37298346
[   27222 Epoch:[24/25] [ 989/1093] time: 2998.4051, d_loss: 1.24010205, g_loss: 2.31146598, walk_loss: 2.31146574
[   27223 Epoch:[24/25] [ 990/1093] time: 2998.4876, d_loss: 1.24611700, g_loss:

[   27287 Epoch:[24/25] [1054/1093] time: 3004.2644, d_loss: 1.23527598, g_loss: 2.25647354, walk_loss: 2.25647306
[   27288 Epoch:[24/25] [1055/1093] time: 3004.3563, d_loss: 1.21839929, g_loss: 2.40575194, walk_loss: 2.40575218
[   27289 Epoch:[24/25] [1056/1093] time: 3004.4482, d_loss: 1.19632185, g_loss: 2.29593420, walk_loss: 2.29593420
[   27290 Epoch:[24/25] [1057/1093] time: 3004.5398, d_loss: 1.24126101, g_loss: 2.38946033, walk_loss: 2.38946033
[   27291 Epoch:[24/25] [1058/1093] time: 3004.6316, d_loss: 1.28721929, g_loss: 2.24930811, walk_loss: 2.24930763
[   27292 Epoch:[24/25] [1059/1093] time: 3004.7216, d_loss: 1.22529542, g_loss: 2.29413152, walk_loss: 2.29413152
[   27293 Epoch:[24/25] [1060/1093] time: 3004.8086, d_loss: 1.21021926, g_loss: 2.29108405, walk_loss: 2.29108381
[   27294 Epoch:[24/25] [1061/1093] time: 3004.8977, d_loss: 1.22228503, g_loss: 2.29793835, walk_loss: 2.29793811
[   27295 Epoch:[24/25] [1062/1093] time: 3004.9881, d_loss: 1.23219967, g_loss:

SystemExit: 

/afs/csail.mit.edu/u/j/jahanian/.conda/envs/myenv/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3275: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
import math
def conv_out_size_same(size, stride):
  return int(math.ceil(float(size) / float(stride)))

In [ ]:
s_h, s_w = 28, 28
s_h2, s_w2 = conv_out_size_same(s_h, 2), conv_out_size_same(s_w, 2)
s_h4, s_w4 = conv_out_size_same(s_h2, 2), conv_out_size_same(s_w2, 2)
s_h8, s_w8 = conv_out_size_same(s_h4, 2), conv_out_size_same(s_w4, 2)
s_h16, s_w16 = conv_out_size_same(s_h8, 2), conv_out_size_same(s_w8, 2)
print(s_h, s_h2, s_h4, s_h8, s_h16)

In [ ]:
!tensorboard --logdir ./out/ckpt/logs

TensorBoard 1.12.2 at http://visiongpu18:6006 (Press CTRL+C to quit)
W0907 19:42:19.440288 ProjectorPluginIsActiveThread checkpoint_management.py:285] FailedPreconditionError: /afs/csail.mit.edu/u/j/jahanian/ibig/ibig_git/ibiggan/gan_workspace/DCGAN-tensorflow/out/ckpt/checkpoint; Is a directory
W0907 19:42:19.440288 139755459168000 checkpoint_management.py:285] FailedPreconditionError: /afs/csail.mit.edu/u/j/jahanian/ibig/ibig_git/ibiggan/gan_workspace/DCGAN-tensorflow/out/ckpt/checkpoint; Is a directory
W0907 19:42:19.442316 ProjectorPluginIsActiveThread checkpoint_management.py:286] /afs/csail.mit.edu/u/j/jahanian/ibig/ibig_git/ibiggan/gan_workspace/DCGAN-tensorflow/out/ckpt/logs/../checkpoint: Checkpoint ignored
W0907 19:42:19.442316 139755459168000 checkpoint_management.py:286] /afs/csail.mit.edu/u/j/jahanian/ibig/ibig_git/ibiggan/gan_workspace/DCGAN-tensorflow/out/ckpt/logs/../checkpoint: Checkpoint ignored
W0907 19:42:19.752468 ProjectorPluginIsActiveThread checkpoint_management